In [1]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.datasets import load_wine
import optuna
from optuna.samplers import TPESampler
import pickle
import pandas as pd 
import joblib
import os, sys 
sys.path.append('/global/cfs/projectdirs/atlas/hrzhao/HEP_Repo/QG_Calibration/BDT_EB4/LightGBM')
from LightGBM_BDT_train import *


In [28]:
use_full_dataset = 0

if use_full_dataset:
    sample_path = '/global/cfs/projectdirs/atlas/hrzhao/HEP_Repo/QG_Calibration/BDT_EB4/samples/sample_all_jets.pkl'
    output_path = './full_dataset'
    n_trails = 100
    
else:
    sample_path = '/global/cfs/projectdirs/atlas/hrzhao/HEP_Repo/QG_Calibration/BDT_EB4/samples/sample_testweight_123'
    output_path = "./small_dataset"
    n_trails = 10
# Use for full dataset tuning
# sample_path = '/global/cfs/projectdirs/atlas/hrzhao/HEP_Repo/QG_Calibration/BDT_EB4/samples/sample_all_jets.pkl'

# Use for code dev with a small dataset 

output_folder = Path(output_path)
if not output_folder.exists():
    output_folder.mkdir(parents=True, exist_ok=True)

study_output = output_folder / 'study.pkl'
gbdt_filename = output_folder / 'lightgbm_gbdt.pkl'
eval_result_filename = output_folder / 'eval_result.pkl'

training_vars = ['jet_pt', 'jet_nTracks', 'jet_trackWidth', 'jet_trackC1']
training_weight = ['flatpt_weight']

label_pt_bin = [500, 600, 800, 1000, 1200, 1500, 2000]


In [3]:
sample = pd.read_pickle(sample_path)

In [4]:
sample.head()

,jet_pt,jet_eta,jet_nTracks,jet_trackWidth,jet_trackC1,jet_trackBDT,jet_PartonTruthLabelID,equal_weight,event_weight,flatpt_weight,is_forward,pt_idx,target
3040225,1771.308105,-1.185907,24.0,0.111155,0.297679,-0.001197,1.0,1.0,0.002129,1.832609e-06,1.0,5,0.0
3308762,576.650024,-1.632921,18.0,0.030110,0.227651,0.115335,21.0,1.0,0.954678,4.073045e-07,1.0,0,1.0
24589109,549.962463,-0.248085,12.0,0.028959,0.142942,-0.211451,4.0,1.0,0.534956,1.849374e-07,0.0,0,0.0
6893257,926.854065,0.773220,11.0,0.022511,0.151775,-0.290329,2.0,1.0,0.130191,7.915768e-07,1.0,2,0.0
729798,1347.683838,-0.424933,20.0,0.046835,0.199852,-0.123005,2.0,1.0,0.039094,3.223083e-06,1.0,4,0.0


In [5]:
X = sample.iloc[:, :-1]

target_idx = sample.columns.get_loc('target')
y = sample.iloc[:, target_idx]

X_dev,X_test, y_dev,y_test = train_test_split(X, y, test_size=0.1, random_state=456)
X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=0.1/0.9, random_state=789)

## Train a base model

In [6]:
# After test, eval_sample_weight already pass the event weight 
# def auc_weighted(y_true, y_pred, weight):
#     fpr, tpr, thresholds = roc_curve(y_true, y_pred, sample_weight = weight)
#     roc_auc = auc(fpr, tpr)

#     return ('auc_weighted', eval_result, is_higher_better)

In [7]:
#### Train a base model with default setting. Only set random_state for reproducibility. 
base_model = lgb.LGBMClassifier(random_state=42)
# base_model.fit(X_train[training_vars], y_train, sample_weight=X_train[training_weight].to_numpy().flatten())
base_model.fit(X = X_train[training_vars], y = y_train, sample_weight=X_train[training_weight].to_numpy().flatten(),
               eval_set = [(X_val[training_vars], y_val)], eval_sample_weight = [X_val["event_weight"].to_numpy().flatten()],
               eval_metric = ['binary_logloss', 'auc'])

[1]	valid_0's binary_logloss: 0.687298	valid_0's auc: 0.809738
[2]	valid_0's binary_logloss: 0.658713	valid_0's auc: 0.815083
[3]	valid_0's binary_logloss: 0.635221	valid_0's auc: 0.816445
[4]	valid_0's binary_logloss: 0.61627	valid_0's auc: 0.817139
[5]	valid_0's binary_logloss: 0.600877	valid_0's auc: 0.818534
[6]	valid_0's binary_logloss: 0.58712	valid_0's auc: 0.819185
[7]	valid_0's binary_logloss: 0.575876	valid_0's auc: 0.819846
[8]	valid_0's binary_logloss: 0.567007	valid_0's auc: 0.8206
[9]	valid_0's binary_logloss: 0.559179	valid_0's auc: 0.821061
[10]	valid_0's binary_logloss: 0.552635	valid_0's auc: 0.821239
[11]	valid_0's binary_logloss: 0.547141	valid_0's auc: 0.822448
[12]	valid_0's binary_logloss: 0.541907	valid_0's auc: 0.823068
[13]	valid_0's binary_logloss: 0.537786	valid_0's auc: 0.823633
[14]	valid_0's binary_logloss: 0.533981	valid_0's auc: 0.823826
[15]	valid_0's binary_logloss: 0.530847	valid_0's auc: 0.824195
[16]	valid_0's binary_logloss: 0.527931	valid_0's auc

LGBMClassifier(random_state=42)

In [8]:
y_test_pred = base_model.predict(X_test[training_vars])
accuracy_score(y_test, y_test_pred)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         0.0       0.78      0.79      0.78    283334
         1.0       0.72      0.71      0.71    216666

    accuracy                           0.75    500000
   macro avg       0.75      0.75      0.75    500000
weighted avg       0.75      0.75      0.75    500000



In [9]:
y_train_decisions = base_model.predict(X_train[training_vars], raw_score = True)
y_test_decisions = base_model.predict(X_test[training_vars], raw_score = True)

In [10]:
plot_decision_func(X_test, y_test_decisions, y_test, output_path)

In [11]:
plot_ROC(y_decisions=y_test_decisions, y_tmva=X_test.iloc[:,X_test.columns.get_loc('jet_trackBDT')], 
                 y_ntrk=X_test.iloc[:,X_test.columns.get_loc('jet_nTracks')], target=y_test, 
                 X_weight=X_test['event_weight'], features=" 4 vars", output_path=output_path)


In [12]:
plot_overtraining_validation(X_dev=X_train, X_test=X_test, y_dev=y_train, y_test=y_test, 
                                    y_dev_decisions=y_train_decisions, y_test_decisions=y_test_decisions, 
                                    output_path=output_path)

## Tune the model

In [13]:
def objective(trial):
    """
    Objective function to be minimized.
    """
    param = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    # TODO Add a pruner to observe intermediate results and stop unpromising trials.
    # https://github.com/optuna/optuna-examples/blob/main/lightgbm/lightgbm_integration.py 
    # https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/003_efficient_optimization_algorithms.html
    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc")
    
    gbm = lgb.LGBMClassifier(**param)
    # gbm.fit(X = X_train[training_vars], y = y_train, sample_weight=X_train[training_weight].to_numpy().flatten())
    gbm.fit(X = X_train[training_vars], y = y_train, sample_weight=X_train[training_weight].to_numpy().flatten(),
            eval_set = [(X_val[training_vars], y_val)], eval_sample_weight = [X_val["event_weight"].to_numpy().flatten()],
            callbacks=[pruning_callback])

    # choose the highest auc score with event weight  
    y_val_decisions = gbm.predict(X_val[training_vars], raw_score = True)
    fpr, tpr, thresholds = roc_curve(y_val, y_val_decisions, sample_weight = X_val['event_weight'])
    roc_auc = auc(fpr, tpr)

    return roc_auc


In [14]:
sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="lightgbm_qgtagging", direction="maximize", sampler = sampler, pruner=optuna.pruners.HyperbandPruner())

study.optimize(objective, n_trials=n_trails, n_jobs = 1)
# n_jobs doesn't make tuning faster because it uses multi-threading.
# https://optuna.readthedocs.io/en/stable/faq.html#multi-threading-parallelization-with-a-single-node

[I 2023-01-24 12:42:33,904] A new study created in memory with name: lightgbm_qgtagging


[LightGBM] [Warning] bagging_fraction is set=0.48805353449026784, subsample=1.0 will be ignored. Current value: bagging_fraction=0.48805353449026784
[LightGBM] [Warning] lambda_l1 is set=5.6649755830282306e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.6649755830282306e-05
[LightGBM] [Warning] feature_fraction is set=0.5813995435791038, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5813995435791038
[LightGBM] [Warning] lambda_l2 is set=0.030403280126677572, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.030403280126677572
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-01-24 12:42:58,873] Trial 0 finished with value: 0.8147198216905984 and parameters: {'lambda_l1': 5.6649755830282306e-05, 'lambda_l2': 0.030403280126677572, 'num_leaves': 2, 'feature_fraction': 0.5813995435791038, 'bagging_fraction': 0.48805353449026784, 'bagging_freq': 1, 'min_child_samples': 22}. Best is trial 0 with value: 0.8147198216905984.


[LightGBM] [Warning] bagging_fraction is set=0.8111317002380557, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8111317002380557
[LightGBM] [Warning] lambda_l1 is set=1.2883876209377052e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.2883876209377052e-05
[LightGBM] [Warning] feature_fraction is set=0.6515167086419769, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6515167086419769
[LightGBM] [Warning] lambda_l2 is set=3.72312200494449e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.72312200494449e-05
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


[I 2023-01-24 12:44:39,977] Trial 1 finished with value: 0.8303278482942958 and parameters: {'lambda_l1': 1.2883876209377052e-05, 'lambda_l2': 3.72312200494449e-05, 'num_leaves': 139, 'feature_fraction': 0.6515167086419769, 'bagging_fraction': 0.8111317002380557, 'bagging_freq': 2, 'min_child_samples': 89}. Best is trial 1 with value: 0.8303278482942958.


...

[I 2023-01-24 12:48:15,493] Trial 6 finished with value: 0.825681903485009 and parameters: {'lambda_l1': 3.341919070318744e-06, 'lambda_l2': 0.12691529280491062, 'num_leaves': 28, 'feature_fraction': 0.6687361157055431, 'bagging_fraction': 0.9451573018558573, 'bagging_freq': 3, 'min_child_samples': 32}. Best is trial 1 with value: 0.8303278482942958.


[LightGBM] [Warning] bagging_fraction is set=0.5593279956233357, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5593279956233357
[LightGBM] [Warning] lambda_l1 is set=1.4799844388224288e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4799844388224288e-07
[LightGBM] [Warning] feature_fraction is set=0.5269768696000354, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5269768696000354
[LightGBM] [Warning] lambda_l2 is set=1.493834966470408e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.493834966470408e-08
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


[I 2023-01-24 12:48:20,507] Trial 7 pruned. Trial was pruned at iteration 3.
[I 2023-01-24 12:49:14,686] Trial 8 finished with value: 0.8282097133198034 and parameters: {'lambda_l1': 0.0014691239860705116, 'lambda_l2': 2.091978294467618e-07, 'num_leaves': 152, 'feature_fraction': 0.8198550160125587, 'bagging_fraction': 0.46140065729669555, 'bagging_freq': 3, 'min_child_samples': 71}. Best is trial 1 with value: 0.8303278482942958.


[LightGBM] [Warning] bagging_fraction is set=0.7089334672349852, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7089334672349852
[LightGBM] [Warning] lambda_l1 is set=5.340887446119036e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.340887446119036e-05
[LightGBM] [Warning] feature_fraction is set=0.7982767871318732, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7982767871318732
[LightGBM] [Warning] lambda_l2 is set=2.8156659603215856e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.8156659603215856e-08
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7


[I 2023-01-24 12:50:39,189] Trial 9 finished with value: 0.8302146549813526 and parameters: {'lambda_l1': 5.340887446119036e-05, 'lambda_l2': 2.8156659603215856e-08, 'num_leaves': 138, 'feature_fraction': 0.7982767871318732, 'bagging_fraction': 0.7089334672349852, 'bagging_freq': 7, 'min_child_samples': 61}. Best is trial 1 with value: 0.8303278482942958.


In [17]:
import joblib
joblib.dump(study, study_output)

['small_dataset/study.pkl']

In [ ]:
# study = joblib.load("study.pkl")

In [18]:
study.best_params

{'lambda_l1': 1.2883876209377052e-05,
 'lambda_l2': 3.72312200494449e-05,
 'num_leaves': 139,
 'feature_fraction': 0.6515167086419769,
 'bagging_fraction': 0.8111317002380557,
 'bagging_freq': 2,
 'min_child_samples': 89}

In [19]:
study.best_value

0.8303278482942958

## Use the best params to train

In [22]:
eval_result={}
best_model = lgb.LGBMClassifier(**study.best_params)
best_model.fit(X = X_train[training_vars], y = y_train, sample_weight=X_train[training_weight].to_numpy().flatten(),
               eval_set = [(X_val[training_vars], y_val)], eval_sample_weight = [X_val["event_weight"].to_numpy().flatten()],
               eval_metric = ['binary_logloss', 'auc'], callbacks=[lgb.early_stopping(5), lgb.record_evaluation(eval_result)])

Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.505323	valid_0's auc: 0.830264


LGBMClassifier(random_state=42)

In [29]:
joblib.dump(best_model, gbdt_filename)
joblib.dump(eval_result, eval_result_filename)

['small_dataset/eval_result.pkl']